In [1]:
import re
import pandas as pd
import time
import socket
socket.setdefaulttimeout(5)

pd.options.display.max_columns = None
pd.options.display.max_rows = None

pd.set_option('display.max_colwidth', 1000)

from bs4 import BeautifulSoup
from urllib.parse import quote
import urllib.request as req

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,}

#
# Function to get the soup objects for a given URL
#
def getSoup(url):
    try:
        request= req.Request(url,None,headers)
        response = req.urlopen(request) 
        data = response.read()
        soup = BeautifulSoup(data, "html.parser")

    except Exception as e:
        print("There was an error retrieving", url, e)  
        
    finally:
        response.close()
        
    return soup
def cleanText(text):
    return re.sub(r'\n','',text)

In [5]:
# top250_df_copy = top250_df_copy[0:100]
top250_df = pd.DataFrame(columns=['Title','Rating','IMDB_MAIN_URL'])# Initialize an empty dataframe
list_home_url = 'http://www.imdb.com/search/title?groups=top_1000&sort=user_rating&start=@@@&view=simple'
for i in range(1,1000,100):
    pagesoup = getSoup(re.sub('@@@','1',list_home_url))
    rows = pagesoup.findAll('tr',{'class':re.compile('even|odd')})
    for row in rows[0:10]:
        try:
            movie_name = row.find('a').text
            movie_url = url_start+row.find('a')['href']
            rating = row.find('b').text
            top250_df.loc[len(top250_df)] = [movie_name,rating,movie_url]
        except Exception as e:
            print(row.text, movie_url,e)


year_list = []
all_genre_list = []
awards_url_list = []
genre_list = []
year = 0
for index, row in top250_df_copy.iterrows():
    try:
        temp_url = row['IMDB_MAIN_URL']
        oneMovie = getSoup(temp_url)
        year_soup = oneMovie.find('span', {'id': 'titleYear'})
        year = re.sub(r'[()]', "", year_soup.text)
        genre_soup = oneMovie.findAll('span', {'itemprop': 'genre'})
        genre_list = []
        for genre in genre_soup:
            genre_list.append(genre.text)
        awards_soup = oneMovie.find(text='See more awards')
        if (awards_soup):
            moreawards_url = url_start + awards_soup.parent['href']

    except Exception as e:
        print("Error at stpe 1", e)
        continue

    finally:
        year_list.append(year)
        all_genre_list.append(' '.join(
            genre_list))  # Using a lit here. Maybe put them in different columns? Dont know if that would be useful
        awards_url_list.append(moreawards_url)
        moreawards_url = " "  # Cleaning the variable in case the next movie does not have an awards page
        time.sleep(2)
        if (index % 25 == 0): print("Fetched till ", index)

top250_df_copy.insert(loc=len(top250_df_copy.columns), column='YEAR', value=year_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='Genre', value=all_genre_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='IMDB_AWARDS_URL', value=awards_url_list)

top250_df_copy.to_csv("/Users/gautamborgohain/Desktop/step1_imdb.csv")

oscar_nominations_list = []
oscar_wins_list = []
gg_nominations_list = []
gg_wins_list = []
sag_nominations_list = []
sag_wins_list = []
bafta_nominations_list = []
bafta_wins_list = []
cc_wins_list = []
cc_nominations_list = []
total_nominations_list = []
total_wins_list = []


def getCount(td):
    return td.get('rowspan') if (td.get('rowspan')) else 0


def toNumber(numberlist):
    return numberlist[0] if (numberlist) else 0


for index, row in top250_df_copy.iterrows():
    temp_url = row['IMDB_AWARDS_URL']
    if (temp_url != ' '):
        oscar_win = 0
        oscar_nom = 0
        BAFTA_win = 0
        BAFTA_nom = 0
        gg_win = 0
        gg_nom = 0
        sag_win = 0
        sag_nom = 0
        cc_win = 0
        cc_nom = 0
        try:
            temp_soup = getSoup(temp_url)
            allstats = temp_soup.find('div', {'class': 'desc'})
            total_wins = toNumber(re.findall(r'([0-9]*) wins', allstats.text))
            total_nominations = toNumber(re.findall(r'([0-9]*) nominations', allstats.text))
            td_award = temp_soup.findAll('td', {'class': 'title_award_outcome'})
            for td in td_award:
                if 'Won\nOscar' in td.text:
                    oscar_win = getCount(td)
                elif 'Nominated\nOscar' in td.text:
                    oscar_nom = getCount(td)
                elif 'Won\nBAFTA' in td.text:
                    BAFTA_win = getCount(td)
                elif 'Nominated\nBAFTA' in td.text:
                    BAFTA_nom = getCount(td)
                elif 'Won\nGolden Globe' in td.text:
                    gg_win = getCount(td)
                elif 'Nominated\nGolden Globe' in td.text:
                    gg_nom = getCount(td)
                elif 'Won\nActor' in td.text:
                    sag_win = getCount(td)
                elif 'Nominated\nActor' in td.text:
                    sag_nom = getCount(td)
                elif 'Won\nCritics Choice' in td.text:
                    cc_win = getCount(td)
                elif 'Nominated\nCritics Choice' in td.text:
                    cc_nom = getCount(td)
        except Exception as e:
            print("Error at step 2", e)
            continue

        finally:
            oscar_wins_list.append(oscar_win)
            oscar_nominations_list.append(oscar_nom)
            cc_wins_list.append(cc_win)
            cc_nominations_list.append(cc_nom)
            bafta_wins_list.append(BAFTA_win)
            bafta_nominations_list.append(BAFTA_nom)
            gg_wins_list.append(gg_win)
            gg_nominations_list.append(gg_nom)
            sag_wins_list.append(sag_win)
            sag_nominations_list.append(sag_nom)
            total_nominations_list.append(total_nominations)
            total_wins_list.append(total_wins)
            if (index % 25 == 0): print("Fetched till ", index)

movies_withoutAwards = top250_df_copy.loc[top250_df_copy.IMDB_AWARDS_URL == ' ']
top250_df_copy = top250_df_copy.loc[
    top250_df_copy.IMDB_AWARDS_URL != ' ']  # Remove the movies that did not have awards page
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='TOTAL_NOM', value=total_nominations_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='TOTAL_WINS', value=total_wins_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='OSCAR_NOM', value=oscar_nominations_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='OSCAR_WIN', value=oscar_wins_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='GG_NOM', value=gg_nominations_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='GG_WINS', value=gg_wins_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='BAFTA_NOM', value=bafta_nominations_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='BAFTA_WIN', value=bafta_wins_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='SAG_NOM', value=sag_nominations_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='SAG_WINS', value=sag_wins_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='CC_NOM', value=cc_nominations_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='CC_WINS', value=cc_wins_list)

top250_df_copy.to_csv("/Users/gautamborgohain/Desktop/step2_imdb.csv")

metaSearch_Start_url = 'http://www.metacritic.com/search/movie/'
metaSearch_End_url = "/results"
meta_home = "http://www.metacritic.com/"
critic_score_list = []
user_score_list = []
movie_url_list = []
for index, row in top250_df_copy.iterrows():
    critic_score = 0
    user_score = 0
    movie_name = row['Title']
    movie_name_encoded = quote(movie_name.encode('utf8'))
    metaSearch_url = metaSearch_Start_url + movie_name_encoded + metaSearch_End_url
    try:
        results_soup = getSoup(metaSearch_url)
        time.sleep(2)
        firstResult = results_soup.find(text=movie_name)  # Search the results for the exact movie name
        if (firstResult):
            movie_home_url_part = firstResult.parent['href']
            movie_home_url = meta_home + movie_home_url_part
            movie_soup = getSoup(movie_home_url)  # Go to the movie's metacritic homepage
            critic_shell = movie_soup.findAll('a', {
                'class': 'metascore_anchor'})  # This gives a list of all the scores, we need the first two
            if (critic_shell):
                critic_score = cleanText(critic_shell[0].text)
                user_score = cleanText(critic_shell[1].text)
    except Exception as e:
        print("Error at step 3", e)
        continue


    finally:
        movie_url_list.append(metaSearch_url)
        critic_score_list.append(critic_score)
        user_score_list.append(user_score)
        time.sleep(2)
        if (index % 25 == 0): print("Fetched till ", index)

top250_df_copy.insert(loc=len(top250_df_copy.columns), column='MC_MOVIE_URL', value=movie_url_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='MC_CRITIC_RATING', value=critic_score_list)
top250_df_copy.insert(loc=len(top250_df_copy.columns), column='MC_USER_RATING', value=user_score_list)

top250_df_copy.to_csv("/Users/gautamborgohain/Desktop/step3_mc.csv")

rotten_home = "http://www.rottentomatoes.com"
rotten_search_start = "http://www.rottentomatoes.com/search/?search="
critic_score_list = []
user_score_list = []
movie_url_list = []

def getMovieURL(movie_name,results_soup):
    ul = results_soup.find('ul',{'id': 'movie_results_ul'})
    if(ul):
        lis = ul.findAll('li',{'class':'media bottom_divider clearfix'})
        for li in lis:
            if movie_name.lower() in li.text.lower().encode('ascii','ignore').decode('utf8'):
                return li.find('a')['href']
    return False

for index, row in top250_df_copy.iterrows():
    critic_score = 0
    user_score = 0
    movie_name = row['Title'].encode('ascii','ignore').decode('utf8')
    movie_name_encoded = re.sub(' ','+',movie_name)
    rtSearch_url = rotten_search_start+movie_name_encoded
    results_soup = getSoup(rtSearch_url)
    time.sleep(2)
    if(results_soup):
        resultsdiv = results_soup.findAll('div',{'id':'scoreStats'})
        if(resultsdiv): # If it went to the Home Page directly
            text = resultsdiv[0].text
            critic_rating = toNumber(re.findall(r'Average Rating:  ([^/]+)',text))
            resultsdiv = results_soup.findAll('div',{'class':'audience-info hidden-xs superPageFontColor'})
            text = resultsdiv[0].text
            user_rating = toNumber(re.findall(r'Average Rating: ([^/]+)',text))
            critic_score_list.append(critic_rating)
            user_score_list.append(user_rating)
            movie_url_list.append(rtSearch_url)
        else:
            home_url = getMovieURL(movie_name,results_soup)
            if(home_url):
                complete_home_url = rotten_home+home_url
                results_list= getSoup(complete_home_url)
                resultsdiv = results_list.findAll('div',{'id':'scoreStats'})
                if(len(resultsdiv)>0):
                    text = resultsdiv[0].text
                    critic_rating = toNumber(re.findall(r'Average Rating:  ([^/]+)',text))
                    resultsdiv = results_list.findAll('div',{'class':'audience-info hidden-xs superPageFontColor'})
                    text = resultsdiv[0].text
                    user_rating = toNumber(re.findall(r'Average Rating: ([^/]+)',text))
                    critic_score_list.append(critic_rating)
                    user_score_list.append(user_rating)
                    movie_url_list.append(complete_home_url)
                else:
                    print("Didnt find ratings info", resultsdiv,complete_home_url)
                    critic_score_list.append(0)
                    user_score_list.append(0)
                    movie_url_list.append(complete_home_url)
            else:
                print("Movie not in the result list", home_url,rtSearch_url)
                critic_score_list.append(0)
                user_score_list.append(0)
                movie_url_list.append(rtSearch_url)
                
    time.sleep(2)
    if(index%25==0):print("Fetched till ",index)

top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='RT_MOVIE_URL',value = movie_url_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='RT_CRITIC_RATING',value = critic_score_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='RT_USER_RATING',value = user_score_list)


Fetched till  0
Fetched till  25
Fetched till  50
Fetched till  75
Fetched till  100
Fetched till  125
Fetched till  150
There was an error retrieving http://www.imdb.com/title/tt2082197/ timed out
Error at stpe 1 local variable 'response' referenced before assignment
Fetched till  175
Fetched till  200
Fetched till  225
Fetched till  250
Fetched till  275
Fetched till  300
Fetched till  325
Fetched till  350
Fetched till  375
Fetched till  400
Fetched till  425
Fetched till  450
Fetched till  475
Fetched till  500
Fetched till  525
Fetched till  550
Fetched till  575
Fetched till  600
Fetched till  625
Fetched till  650
Fetched till  675
Fetched till  700
Fetched till  725
Fetched till  750
Fetched till  775
Fetched till  800
Fetched till  825
Fetched till  850
Fetched till  875
Fetched till  900
Fetched till  925
Fetched till  950
Fetched till  975
Fetched till  0
Fetched till  25
Fetched till  50
Fetched till  75
Fetched till  100
Fetched till  125
Fetched till  150
Fetched till  17